In [1]:
import random
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import layers, Sequential, Input
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import Dense, LSTM, Concatenate, Input, Conv1D, Flatten, Dropout, BatchNormalization, \
    GlobalAveragePooling1D, MaxPooling1D, GlobalAveragePooling1D, UpSampling1D, concatenate, Conv1DTranspose, \
    Cropping1D, UpSampling1D, ZeroPadding1D, Reshape, Lambda, Multiply, Softmax, Add, Average, Maximum, Multiply, MultiHeadAttention, LayerNormalization
from keras.models import Sequential, Model

matplotlib.use('TkAgg')

In [119]:
data = pd.read_csv("data/DataBase_RF_bi_lowperc_1.csv")

In [120]:
data

,name,Al,Si,Cation #1,Cation #2,Radius #1,Radius #2,Polarizability #1,Polarizability #2,Charge #1,Charge #2,S1 #1,S1' #1,S2 #1,S3 #1,S1 #2,S1' #2,S2 #2,S3 #2
0,Structure: Al_80_Si_112_Boulfelfel_Ca_1_K_78,80,112,1,78,1.00,1.38,22.80,43.40,2,1,0,0,1,0,16,7,34,21
1,Structure: Al_80_Si_112_Boulfelfel_Ca_1_Na_78,80,112,1,78,1.00,1.02,22.80,24.11,2,1,0,0,1,0,1,25,30,22
2,Structure: Al_80_Si_112_Boulfelfel_Ca_1_Rb_78,80,112,1,78,1.00,1.52,22.80,47.30,2,1,1,0,0,0,14,8,35,21
3,Structure: Al_80_Si_112_Boulfelfel_Cs_1_K_79,80,112,1,79,1.67,1.38,59.42,43.40,1,1,0,0,1,0,16,8,35,20
4,Structure: Al_80_Si_112_Boulfelfel_Cs_1_Li_79,80,112,1,79,1.67,0.76,59.42,24.33,1,1,0,0,0,1,0,23,31,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Structure: Al_96_Si_96_Boulfelfel_Rb_14_Na_82,96,96,14,82,1.52,1.02,47.30,24.11,1,1,0,0,0,14,0,32,32,18
496,Structure: Al_96_Si_96_Boulfelfel_Rb_15_Na_81,96,96,15,81,1.52,1.02,47.30,24.11,1,1,0,0,1,14,0,32,31,18
497,Structure: Al_96_Si_96_Boulfelfel_Rb_17_K_79,96,96,17,79,1.52,1.38,47.30,43.40,1,1,2,2,4,9,14,5,35,25
498,Structure: Al_96_Si_96_Boulfelfel_Rb_4_Li_92,96,96,4,92,1.52,0.76,47.30,24.33,1,1,0,0,1,3,0,32,31,29


In [126]:
for index, row in data.iterrows():
    data.loc[len(data)] = [row["name"], row["Al"], row["Si"],
                row["Cation #2"], row["Cation #1"],
                row["Radius #2"], row["Radius #1"],
                row["Polarizability #2"],  row["Polarizability #1"],
                row["Charge #2"], row["Charge #1"],
                row["S1 #2"], row["S1\' #2"], row["S2 #2"], row["S3 #2"],
                row["S1 #1"], row["S1\' #1"], row["S2 #1"], row["S3 #1"]]

In [127]:
data

,name,Al,Si,Cation #1,Cation #2,Radius #1,Radius #2,Polarizability #1,Polarizability #2,Charge #1,Charge #2,S1 #1,S1' #1,S2 #1,S3 #1,S1 #2,S1' #2,S2 #2,S3 #2
0,Structure: Al_80_Si_112_Boulfelfel_Ca_1_K_78,80,112,1,78,1.00,1.38,22.80,43.40,2,1,0,0,1,0,16,7,34,21
1,Structure: Al_80_Si_112_Boulfelfel_Ca_1_Na_78,80,112,1,78,1.00,1.02,22.80,24.11,2,1,0,0,1,0,1,25,30,22
2,Structure: Al_80_Si_112_Boulfelfel_Ca_1_Rb_78,80,112,1,78,1.00,1.52,22.80,47.30,2,1,1,0,0,0,14,8,35,21
3,Structure: Al_80_Si_112_Boulfelfel_Cs_1_K_79,80,112,1,79,1.67,1.38,59.42,43.40,1,1,0,0,1,0,16,8,35,20
4,Structure: Al_80_Si_112_Boulfelfel_Cs_1_Li_79,80,112,1,79,1.67,0.76,59.42,24.33,1,1,0,0,0,1,0,23,31,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Structure: Al_96_Si_96_Boulfelfel_Rb_14_Na_82,96,96,82,14,1.02,1.52,24.11,47.30,1,1,0,32,32,18,0,0,0,14
996,Structure: Al_96_Si_96_Boulfelfel_Rb_15_Na_81,96,96,81,15,1.02,1.52,24.11,47.30,1,1,0,32,31,18,0,0,1,14
997,Structure: Al_96_Si_96_Boulfelfel_Rb_17_K_79,96,96,79,17,1.38,1.52,43.40,47.30,1,1,14,5,35,25,2,2,4,9
998,Structure: Al_96_Si_96_Boulfelfel_Rb_4_Li_92,96,96,92,4,0.76,1.52,24.33,47.30,1,1,0,32,31,29,0,0,1,3


In [4]:
data.iloc[22]

name                 Structure: Al_81_Si_111_Boulfelfel_Cs_2_Li_79
Al                                                              81
Si                                                             111
Cation #1                                                        2
Cation #2                                                       79
Radius #1                                                     1.67
Radius #2                                                     0.76
Polarizability #1                                            59.42
Polarizability #2                                            24.33
Charge #1                                                        1
Charge #2                                                        1
S1 #1                                                            0
S1' #1                                                           0
S2 #1                                                            0
S3 #1                                                         

In [129]:
data = data.loc[(data["Cation #1"] == (data["S1 #1"] + data["S1\' #1"] + data["S2 #1"] + data["S3 #1"]))
                &
                (data["Cation #2"] == (data["S1 #2"] + data["S1\' #2"] + data["S2 #2"] + data[
                    "S3 #2"]))]  #["Cation #1", "S1 #1", "S1\' #1", "S2 #1", "S3 #1"]]

In [128]:
data_without_names = data.drop(columns=['name'])
plt.matshow(data_without_names.corr())
continuous_features = data_without_names.describe().columns
plt.xticks(range(len(continuous_features)), continuous_features)
plt.yticks(range(len(continuous_features)), continuous_features)
plt.colorbar()
plt.show()

In [130]:
features = data.drop(columns=["name", "S1 #1", "S1\' #1", "S2 #1", "S3 #1", "S1 #2", "S1\' #2", "S2 #2", "S3 #2"],
                     axis=1)
labels = data[["S1 #1", "S1\' #1", "S2 #1", "S3 #1", "S1 #2", "S1\' #2", "S2 #2", "S3 #2"]]

In [131]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.25,
                                                                            random_state=0)

In [132]:
features_train

,Al,Si,Cation #1,Cation #2,Radius #1,Radius #2,Polarizability #1,Polarizability #2,Charge #1,Charge #2
53,82,110,2,80,0.76,1.38,24.33,43.40,1,1
170,86,106,6,80,1.38,1.52,43.40,47.30,1,1
661,86,106,85,1,1.52,1.67,47.30,59.42,1,1
366,92,100,82,5,1.52,1.00,47.30,22.80,1,2
477,96,96,17,79,1.38,1.02,43.40,24.11,1,1
...,...,...,...,...,...,...,...,...,...,...
867,92,100,83,9,1.02,1.52,24.11,47.30,1,1
201,87,105,2,85,0.76,1.38,24.33,43.40,1,1
653,85,107,82,3,1.02,1.52,24.11,47.30,1,1
581,83,109,82,1,1.38,1.02,43.40,24.11,1,1


In [92]:
from sklearn.preprocessing import MinMaxScaler

scale_features_mm = MinMaxScaler()
features_train = scale_features_mm.fit_transform(features_train)
features_test = scale_features_mm.transform(features_test)

scale_labels_mm = MinMaxScaler()
labels_train = scale_labels_mm.fit_transform(labels_train)
labels_test = scale_labels_mm.transform(labels_test)

In [93]:
features_train

array([[0.9375    , 0.0625    , 0.08791209, ..., 0.56253413, 0.        ,
        0.        ],
       [0.875     , 0.125     , 0.01098901, ..., 0.66903332, 0.        ,
        0.        ],
       [0.375     , 0.625     , 0.02197802, ..., 0.66903332, 0.        ,
        0.        ],
       ...,
       [0.25      , 0.75      , 0.04395604, ..., 1.        , 0.        ,
        0.        ],
       [0.125     , 0.875     , 0.02197802, ..., 0.04178045, 0.        ,
        0.        ],
       [0.375     , 0.625     , 0.03296703, ..., 0.04178045, 0.        ,
        0.        ]])

In [133]:
def mps_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    delta = tf.abs(y_true - y_pred)
    delta_ = tf.reduce_mean(delta)
    sigma = tf.sqrt(tf.reduce_mean(tf.square(delta - delta_)))
    size = tf.size(delta, out_type=tf.int32)
    return tf.reduce_mean(delta_ / tf.cast(size, tf.float32) * (sigma / tf.cast(size, tf.float32) + 1))


def set_model():
    #shared
    inputs = Input(shape=3, name="struct_input")
    x = Dense(3, activation='relu', name="struct_d1")(inputs)
    x = Dense(2, activation='relu', name="struct_d2")(x)
    feature_ext_model = Model(inputs, x)
    #
    struct1, struct2 = Input(shape=(3,)), Input(shape=(3,))
    out1, out2 = feature_ext_model(struct1), feature_ext_model(struct2)
    y1 = Multiply()([out1, out2])
    y2 = Average()([out1, out2])
    y = Concatenate()([y1, y2])
    #
    y = Dense(10, activation='relu', name="d1")(y2)
    y = Dropout(0.1)(y)
    y = Dense(10, activation='relu', name="d2")(y)
    y = Dropout(0.1)(y)
    y = Dense(8, activation='relu', name="d3")(y)
    model = Model([struct1, struct2], y)
    model.compile(loss="mse", optimizer='Adam')
    return model

def set_model2():
    #
    struct1, struct2 = Input(shape=(3,)), Input(shape=(3,))
    inputs = Concatenate()([struct1, struct2])
    x = Dense(6, activation='relu', name="struct_d1")(inputs)
    x = Dropout(0.2)(x)
    x = Dense(6, activation='relu')(x)
    struct_model = Model([struct1, struct2], x)
    #
    #
    struct_res = Input(shape=(9,))
    x = Dense(9, activation='relu', name="struct_d2")(struct_res)
    x = Dropout(0.2)(x)
    x = Dense(4, activation='relu')(x)
    struct_res_model = Model(struct_res, x)
    #
    input1, input2 = Input(shape=(3,)), Input(shape=(3,))
    num_elements1 = Input(shape=1)
    num_elements2 = Input(shape=1)
    out1, out2 = struct_model([input1, input2]), struct_model([input2, input1])
    y = Average()([out1, out2])
    struct1_state = Concatenate()([y, input1])
    struct2_state = Concatenate()([y, input2])
    struct1_state = struct_res_model(struct1_state)
    struct2_state = struct_res_model(struct2_state)
    
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct1_state)
    output1 = Multiply()([l1, num_elements1])
    l2 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct2_state)
    output2 = Multiply()([l2, num_elements2])
    out = Concatenate()([output1, output2])
    
    model = Model([input1, input2, num_elements1, num_elements2], out)
    model.compile(loss="mse", optimizer='Adam')
    return model


def create_model(x):
    inputs = Input(shape=6)
    num_elements = Input(shape=1)
    x = Dense(10, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    d = Dense(8, activation='relu')(x)
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(d)
    outputs = Multiply()([l1, num_elements])
    model = tf.keras.Model(inputs=[inputs, num_elements], outputs=outputs)
    model.compile(loss="mse", optimizer='Adam')
    return model


#model = create_model(features_train)
model = set_model2()

In [134]:
def generate_data_for_set():
    num_train = [[], []]
    num_test = [[], []]
    f_train = [[], []]
    f_test = [[], []]
    for i in range(len(features_train)):
        f_train[0].append(features_train[["Cation #1", "Radius #1", "Polarizability #1"]].iloc[i])
        f_train[1].append(features_train[["Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
        num_train[0].append(features_train[["Cation #1"]].iloc[i])
        num_train[1].append(features_train[["Cation #2"]].iloc[i])
    for i in range(len(features_test)):
        f_test[0].append(features_test[["Cation #1", "Radius #1", "Polarizability #1"]].iloc[i])
        f_test[1].append(features_test[["Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
        num_test[0].append(features_test[["Cation #1"]].iloc[i])
        num_test[1].append(features_test[["Cation #2"]].iloc[i])

    f_train[0] = np.array(f_train[0])
    f_train[1] = np.array(f_train[1])
    f_test[0] = np.array(f_test[0])
    f_test[1] = np.array(f_test[1])
    
    num_train[0] = np.array(num_train[0])
    num_train[1] = np.array(num_train[1])
    num_test[0] = np.array(num_test[0])
    num_test[1] = np.array(num_test[1])
    return f_train, f_test, num_train, num_test

f_train, f_test, num_train, num_test = generate_data_for_set()

# f_train = []
# f_test = []
# num_train = []
# num_test = []
# for i in range(len(features_train)):
#     #labels_train.iloc[i] /= np.sum(labels_train.iloc[i])
#     f_train.append(features_train[["Cation #1", "Radius #1", "Polarizability #1", "Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
#     num_train.append(features_train.iloc[i]["Al"])
# for i in range(len(features_test)):
#     #labels_test.iloc[i] /= np.sum(labels_test.iloc[i])
#     f_test.append(features_test[["Cation #1", "Radius #1", "Polarizability #1", "Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
#     num_test.append(features_test.iloc[i]["Al"])

In [135]:
def plot():
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plot()


mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath='data/models/mps.keras', save_best_only=True,
                                              monitor='val_loss', mode='min', verbose=1, save_weights_only=False,
                                              save_freq='epoch')

# history = model.fit([np.array(f_train), np.array(num_train)], labels_train,
#                     epochs=100, batch_size=16, shuffle=True,
#                     validation_data=([np.array(f_test), np.array(num_test)], labels_test), callbacks=[mcp_save])

history = model.fit((f_train[0], f_train[1], num_train[0], num_train[1]), labels_train,
                    epochs=500, batch_size=16, shuffle=True,
                    validation_data=((f_test[0], f_test[1], num_test[0], num_test[1]), labels_test), callbacks=[mcp_save])

plot_loss(history)

Epoch 1/500
46/46 [==============================] - ETA: 0s - loss: 133.7994
Epoch 1: val_loss improved from inf to 87.94447, saving model to data/models\mps.keras
46/46 [==============================] - 4s 35ms/step - loss: 133.7994 - val_loss: 87.9445
Epoch 2/500
42/46 [==========================>...] - ETA: 0s - loss: 88.7992
Epoch 2: val_loss improved from 87.94447 to 85.12620, saving model to data/models\mps.keras
46/46 [==============================] - 1s 24ms/step - loss: 89.1387 - val_loss: 85.1262
Epoch 3/500
45/46 [============================>.] - ETA: 0s - loss: 79.0585
Epoch 3: val_loss improved from 85.12620 to 78.33932, saving model to data/models\mps.keras
46/46 [==============================] - 1s 26ms/step - loss: 79.2886 - val_loss: 78.3393
Epoch 4/500
46/46 [==============================] - ETA: 0s - loss: 56.8268
Epoch 4: val_loss improved from 78.33932 to 43.89008, saving model to data/models\mps.keras
46/46 [==============================] - 1s 17ms/step - l

In [136]:
model = tf.keras.models.load_model('data/models/mps.keras')

In [137]:
#predictions = model.predict([np.array(f_test), np.array(num_test)])
predictions = model.predict((f_test[0], f_test[1], num_test[0], num_test[1]))
predictions_2 = model.predict((f_test[1], f_test[0], num_test[1], num_test[0]))

8/8 [==============================] - 0s 6ms/step


In [138]:
print(features_test.iloc[i])
print(labels_test.iloc[i])

Al                    88.00
Si                   104.00
Cation #1             81.00
Cation #2              7.00
Radius #1              0.76
Radius #2              1.52
Polarizability #1     24.33
Polarizability #2     47.30
Charge #1              1.00
Charge #2              1.00
Name: 755, dtype: float64
S1 #1      0
S1' #1    27
S2 #1     31
S3 #1     23
S1 #2      1
S1' #2     0
S2 #2      0
S3 #2      6
Name: 755, dtype: int64


In [155]:
import random


def mps(first, second):
    delta = []
    for i in range(len(first)):
        delta.append(abs(first[i] - second[i]))
    delta_ = sum(delta) / len(delta)
    sigma = 0
    for i in range(len(first)):
        sigma += (delta_ - abs(first[i] - second[i])) ** 2
    sigma = np.sqrt(sigma / len(delta))
    return delta_ / len(delta) * (sigma / len(delta) + 1)


i = random.randint(0, len(predictions) - 1)
print("№", i)
rounded_prediction = [round(p, 1) for p in predictions[i]]
print(rounded_prediction)
rounded_prediction2 = [round(p, 1) for p in predictions_2[i]]
#print(rounded_prediction2)

rounded_real = [round(float(p), 1) for p in np.array(labels_test.iloc[i])]
print(rounded_real)
print(sum(predictions[i]), sum(labels_test.iloc[i]))

№ 77
[0.9, 27.7, 32.4, 27.0, 0.2, 0.1, 0.3, 0.4]
[0.0, 26.0, 32.0, 30.0, 1.0, 0.0, 0.0, 0.0]
89.00000554323196 89


In [141]:
mps_list = []
for i in range(len(predictions)):
    p = predictions[i]
    real = np.array(labels_test.iloc[i])
    mps_list.append(mps(p, real))
print(sum(mps_list))
print(sum(mps_list)/len(mps_list))

50.143496172220004
0.20720452963727273


In [142]:
plt.hist(mps_list)
plt.show()

In [610]:
print(sum(mps_list))

37.59565462806723


In [158]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import textwrap
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau
def corr_heatmap_with_pval(df, method = 'pearson', figsize=(20, 10), title=None):
  """
  df: dataframe to be used. Ensured the dataframe has been sliced to contain only the column you need. It accepts only numerical columns
  method: default uses the pearson method. It overall permits 3 methods; 'pearson', 'spearman' and 'kendall'
  figsize: default is (20, 10) but you can change it based on your preference
  title: Specify the title for your chart, default is None
  """
  # Make a copy of the df
  data = df.copy()
  # Check features correlation
  corr = data.corr(method = method)

  # Create a mask to hide the upper triangle
  mask = np.zeros_like(corr, dtype=bool)
  mask[np.triu_indices_from(mask)] = True

  # Set the diagonal elements of the mask to False to display self-correlation
  np.fill_diagonal(mask, False)

  fig, ax = plt.subplots(figsize=figsize)
  plt.title(title, fontsize=14)

  # Create the heatmap with the custom mask
  heatmap = sns.heatmap(corr,
                        annot=True,
                        annot_kws={"fontsize": 10},  # Adjust annotation font size
                        fmt='.2f',
                        linewidths=0.5,
                        cmap='RdBu',
                        mask=mask,
                        ax=ax)

  # Create a function to calculate and format p-values
  p_values = np.full((corr.shape[0], corr.shape[1]), np.nan)
  for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[1]):
      x = data.iloc[:, i]
      y = data.iloc[:, j]
      mask = ~np.logical_or(np.isnan(x), np.isnan(y))
      if np.sum(mask) > 0:
        if method == 'pearson':
          p_values[i, j] = pearsonr(x[mask], y[mask])[1] #Changes based on the method chosen in the function
        elif method == 'kendall':
          p_values[i, j] = kendalltau(x[mask], y[mask])[1]
        elif method == 'spearman':
          p_values[i, j] = spearmanr(x[mask], y[mask])[1]
  
  p_values = pd.DataFrame(p_values, columns=corr.columns, index=corr.index)

  # Create a mask for the p-values heatmap
  mask_pvalues = np.triu(np.ones_like(p_values), k=1)

  # Calculate the highest and lowest correlation coefficients
  max_corr = np.max(corr.max())
  min_corr = np.min(corr.min())
  
  # Annotate the heatmap with p-values and change text color based on correlation value
  for i in range(p_values.shape[0]):
    for j in range(p_values.shape[1]):
      if mask_pvalues[i, j]:
        p_value = p_values.iloc[i, j]
        if not np.isnan(p_value):
          correlation_value = corr.iloc[i, j]
          text_color = 'white' if correlation_value >= (max_corr - 0.4) or correlation_value <= (min_corr + 0.4) else 'black'
          if p_value <= 0.01:
            #include double asterisks for p-value <= 0.01
            ax.text(i + 0.5, j + 0.8, f'(p = {p_value:.2f})**',
                    horizontalalignment='center',
                    verticalalignment='center',
                    fontsize=8,
                    color=text_color)
          elif p_value <= 0.05:
            #include single asterisks for p-value <= 0.05
            ax.text(i + 0.5, j + 0.8, f'(p = {p_value:.2f})*',
                    horizontalalignment='center',
                    verticalalignment='center',
                    fontsize=8,
                    color=text_color)
          else:
            ax.text(i + 0.5, j + 0.8, f'(p = {p_value:.2f})',
                    horizontalalignment='center',
                    verticalalignment='center',
                    fontsize=8,
                    color=text_color)

  # Customize x-axis labels
  x_labels = [textwrap.fill(label.get_text(), 12) for label in ax.get_xticklabels()]
  ax.set_xticklabels(x_labels, rotation=0, ha="center")

  # Customize y-axis labels
  y_labels = [textwrap.fill(label.get_text(), 15) for label in ax.get_yticklabels()]
  ax.set_yticklabels(y_labels, rotation=0, ha="right")
  ax.grid(False)
  plt.show()

In [161]:
corr_heatmap_with_pval(data_without_names, method = 'kendall', figsize=(10,5), title="Correlation Matrix with P-Value using Function and Kendall Method")